In [37]:
import tensorflow as tf
import numpy as np

print(tf.__version__)
print(np.__version__)

# class BasicBlock(tf.keras.layers.Layer):

#     def __init__(self, filter_num, stride=1):
#       super(BasicBlock, self).__init__()
#       self.conv1 = tf.keras.layers.Conv2D(filters=filter_num,
#                                           kernel_size=(3,3),
#                                           strides=stride,
#                                           padding="same")
#       self.bn1 = tf.keras.layers.BatchNormalization()
#       self.conv2 = tf.keras.layers.Conv2D(filters=filter_num,
#                                           kernel_size=(3,3),
#                                           strides=1,
#                                           padding="same")
#       self.bn2 = tf.keras.layers.BatchNormalization()
#       if stride !=1:
#         self.downsample = tf.keras.sequential()
#         self.downsample.add(tf.keras.layers.Conv2D(filters=filter_num,
#                                                    kernel_size=(1,1),
#                                                    strides=stride))
#         self.downsample.add(tf.keras.layers.BatchNormalization())
#       else:
#         self.downsample = lambda x: x

#     def call(self, inputs, training=None, **kwargs):
#         residual = self.downsample(inputs)

#         x = self.conv(inputs)
#         x = self.bn1(x, training=training)
#         x = tf.nn.relu(x)
#         x = self.conv2(x)
#         x = self.bn2(x, training=training)

#         output = tf.nn.relu(tf.keras.layers.add([residual,x]))

#         return output


# class BottleNeck(tf.keras.layers.Layer):
#     def __init__(self, filter_num, stride=1):
#         super(BottleNeck, self).__init__()
#         self.conv1 = tf.keras.layers.Conv2D(filers=filter_num,
#                                             kernel_size=(1,1),
#                                             strides=1,
#                                             padding='same')
#         self.bn1 = tf.keras.layers.BatchNormalization()
#         self.conv2 = tf.keras.layers.Conv2D(filters=filter_num,
#                                             kernel_size=(3,3),
#                                             strides=stride,
#                                             padding='same')
#         self.bn2 = tf.keras.layers.BatchNormalization()
#         self.conv3 = tf.keras.layers.Conv2D(filters=filter_num *4,
#                                             kernel_size=(1,1),
#                                             strides=1,
#                                             padding='same')
#         self.downsample = tf.keras.Sequential()
#         self.downsample.add(tf.kears.layers.Conv2D(filters=filter_num * 4,
#                                                    kernel_size=(1,1),
#                                                    strides=stride))
#         self.downsample.add(tf.keras.layers.BatchNormalization())

#     def call(self, inputs, training=None, **kwargs):
#       residual = self.downsample(inputs)

#       x = self.conv1(inputs)
#       x = self.bn1(x, training=training)
#       x = tf.nn.relu(x)
#       x = self.conv2(x)
#       x = self.bn2(x, training=training)
#       x = tf.nn.relu(x)
#       x = self.conv3(x)
#       x = self.bn3(x, training=training)

#       output = tf.nn.relu(tf.keras.layers.add([residual, x]))

#       return output

# def make_bottleneck_layer(filter_num, blocks, stride=1):
#     res_block = tf.keras.Sequential()
#     res_block.add(BottleNeck(filter_num, stride=stride))

#     for _ in range(1, blocks):
#         res_block.add(BottleNeck(filter_num, stride=1))

#     return res_block


2.8.0
1.21.5


In [38]:
import tensorflow_datasets as tfds


(ds_train, ds_test), ds_info = tfds.load(
    'cats_vs_dogs',
    split = ('train[:80%]', 'train[80%:]'),
    with_info = True,
    as_supervised = True
)

In [39]:
def normalize_and_resize_img(image, label):
    """Normalizes images: `uint8` -> `float32`."""
    image = tf.image.resize(image, [224, 224])
    return tf.cast(image, tf.float32) / 255., label

In [40]:
ds_train = apply_normalize_on_dataset(ds_train, batch_size=BATCH_SIZE)
ds_test = apply_normalize_on_dataset(ds_test, batch_size=BATCH_SIZE)

In [41]:
def apply_normalize_on_dataset(ds, is_test=False, batch_size=16):
    ds = ds.map(
        normalize_and_resize_img, 
        num_parallel_calls=1
    )
    ds = ds.batch(batch_size)
    if not is_test:
        ds = ds.repeat()
        ds = ds.shuffle(200)
    ds = ds.prefetch(tf.data.experimental.AUTOTUNE)
    return ds

In [42]:
from keras import models, layers
from keras import Input
from keras.models import Model, load_model
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers, initializers, regularizers, metrics
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.layers import BatchNormalization, Conv2D, Activation, Dense, GlobalAveragePooling2D, MaxPooling2D, ZeroPadding2D, Add

import os
import matplotlib.pyplot as plt
import numpy as np
import math

import tensorflow_datasets as tfds 



# Tensorflow 데이터셋을 로드하면 꼭 feature 정보를 확인해 보세요. 
print(ds_info.features)

FeaturesDict({
    'image': Image(shape=(None, None, 3), dtype=tf.uint8),
    'image/filename': Text(shape=(), dtype=tf.string),
    'label': ClassLabel(shape=(), dtype=tf.int64, num_classes=2),
})


In [43]:
# Tensorflow 데이터셋을 로드하면 꼭 feature 정보를 확인해 보세요. 
print(ds_info.features)

FeaturesDict({
    'image': Image(shape=(None, None, 3), dtype=tf.uint8),
    'image/filename': Text(shape=(), dtype=tf.string),
    'label': ClassLabel(shape=(), dtype=tf.int64, num_classes=2),
})


In [44]:
# Tensorflow가 활용할 GPU가 장착되어 있는지 확인해 봅니다.
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [45]:
BATCH_SIZE = 32
EPOCH = 20

In [46]:
# 50
input_tensor = Input(shape=(224, 224, 3), dtype='float32', name='input')


def conv1_layer(x):
    x = ZeroPadding2D(padding=(3,3))(x)
    x = Conv2D(64,(7,7), strides=(2,2))(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = ZeroPadding2D(padding=(1,1))(x)

    return x



In [47]:
def conv2_layer(x):
    x = MaxPooling2D((3,3), 2)(x)

    shortcut = x

    for i in range(3):
      if (i == 0):
        x = Conv2D(64, (1,1), strides=(1,1), padding='same')(x)
        x = BatchNormalization()(x)
        x = Activation('relu')(x)

        x = Conv2D(64,(3,3), strides=(1,1), padding='same')(x)
        x = BatchNormalization()(x)
        x = Activation('relu')(x)

        x = Conv2D(256, (1, 1), strides = (1,1), padding = 'same')(x)
        shortcut = Conv2D(256, (1,1), strides = (1,1), padding = 'same')(shortcut)
        x = BatchNormalization()(x)
        shortcut = BatchNormalization()(shortcut)


        x = Add()([x, shortcut])
        x = Activation('relu')(x)

        shortcut = x

      else: 
          x = Conv2D(64, (1,1), strides=(1,1), padding='same')(x)
          x = BatchNormalization()(x)
          x = Activation('relu')(x)

          x = Conv2D(64, (3,3), strides=(1,1), padding='same')(x)
          x = BatchNormalization()(x)
          x = Activation('relu')(x)

          x = Conv2D(256, (1,1), strides = (1,1), padding = 'same')(x)
          x = BatchNormalization()(x)


          x = Add()([x, shortcut])
          x = Activation('relu')(x)

          shortcut = x

    return x



In [48]:
def conv3_layer(x):
    shortcut = x

    for i in range(4):

        if(i==0):
            x = Conv2D(128, (1,1), strides=(2,2), padding='same')(x)
            x = BatchNormalization()(x)
            x = Activation('relu')(x)

            x = Conv2D(128, (3, 3), strides=(1,1), padding='same')(x)
            x = BatchNormalization()(x)
            x = Activation('relu')(x)

            x = Conv2D(512, (1,1), strides=(1,1), padding='same')(x)
            shortcut = Conv2D(512, (1,1), strides=(2,2), padding='same')(shortcut)
            x = BatchNormalization()(x)
            shortcut = BatchNormalization()(shortcut)

            x = Add()([x, shortcut])
            x = Activation('relu')(x)

            shortcut = x
            
        else:
          x = Conv2D(128, (1,1), strides=(1,1), padding='same')(x)
          x = BatchNormalization()(x)
          x = Activation('relu')(x)

          x = Conv2D(128, (3,3), strides=(1,1), padding='same')(x)
          x = BatchNormalization()(x)
          x = Activation('relu')(x)

          x = Conv2D(512, (1,1), strides=(1,1), padding='same')(x)
          x = BatchNormalization()(x)

          x = Add()([x, shortcut])
          x = Activation('relu')(x)

          shortcut = x

    return x


In [49]:
def conv4_layer(x):
    shortcut = x

    for i in range(6):
        if(i == 0):
            x = Conv2D(256,(1,1), strides=(2,2), padding='valid')(x)
            x = BatchNormalization()(x)
            x = Activation('relu')(x)

            x = Conv2D(256, (3,3), strides=(1,1), padding='same')(x)
            x = BatchNormalization()(x)
            x = Activation('relu')(x)

            x = Conv2D(1024, (1,1), strides = (1,1), padding = 'valid')(x)
            shortcut = Conv2D(1024, (1,1), strides=(2,2), padding='valid')(shortcut)
            x = BatchNormalization()(x)
            shortcut = BatchNormalization()(shortcut)

            x = Add()([x, shortcut])
            x = Activation('relu')(x)

            shortcut = x


        else:
            x = Conv2D(256, (1,1), strides=(1,1), padding='same')(x)
            x = BatchNormalization()(x)
            x = Activation('relu')(x)

            x = Conv2D(256, (3,3), strides=(1,1), padding='same')(x)
            x = BatchNormalization()(x)
            x = Activation('relu')(x)

            x = Conv2D(1024, (1,1), strides=(1,1), padding='same')(x)
            x = BatchNormalization()(x)

            x = Add()([x, shortcut])
            x = Activation('relu')(x)

            shortcut = x

    return x


In [50]:
def conv5_layer(x):
    shortcut = x    
  
    for i in range(3):     
        if(i == 0):            
            x = Conv2D(512, (1, 1), strides=(2, 2), padding='same')(x)
            x = BatchNormalization()(x)
            x = Activation('relu')(x)        
            
            x = Conv2D(512, (3, 3), strides=(1, 1), padding='same')(x)
            x = BatchNormalization()(x)
            x = Activation('relu')(x)  
 
            x = Conv2D(2048, (1, 1), strides=(1, 1), padding='same')(x)
            shortcut = Conv2D(2048, (1, 1), strides=(2, 2), padding='same')(shortcut)
            x = BatchNormalization()(x)
            shortcut = BatchNormalization()(shortcut)            
 
            x = Add()([x, shortcut])  
            x = Activation('relu')(x)      
 
            shortcut = x               
        
        else:
            x = Conv2D(512, (1, 1), strides=(1, 1), padding='same')(x)
            x = BatchNormalization()(x)
            x = Activation('relu')(x)
            
            x = Conv2D(512, (3, 3), strides=(1, 1), padding='same')(x)
            x = BatchNormalization()(x)
            x = Activation('relu')(x)
 
            x = Conv2D(2048, (1, 1), strides=(1, 1), padding='same')(x)
            x = BatchNormalization()(x)           
            
            x = Add()([x, shortcut]) 
            x = Activation('relu')(x)       
 
            shortcut = x                  
 
    return x



In [51]:
x = conv1_layer(input_tensor)
x = conv2_layer(x)
x = conv3_layer(x)
x = conv4_layer(x)
x = conv5_layer(x)

x = GlobalAveragePooling2D()(x)
output_tensor = Dense(2, activation = 'softmax')(x)

resnet50 = Model(input_tensor, output_tensor)



resnet50.summary()



Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input (InputLayer)             [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 zero_padding2d_4 (ZeroPadding2  (None, 230, 230, 3)  0          ['input[0][0]']                  
 D)                                                                                               
                                                                                                  
 conv2d_106 (Conv2D)            (None, 112, 112, 64  9472        ['zero_padding2d_4[0][0]']       
                                )                                                           

In [52]:
resnet50.compile(
    loss='sparse_categorical_crossentropy',
    optimizer=tf.keras.optimizers.SGD(learning_rate=0.01, clipnorm=1.),
    metrics=['accuracy'],
)

history_50 = resnet50.fit(
    ds_train,
    steps_per_epoch=int(ds_info.splits['train[:80%]'].num_examples/BATCH_SIZE),
    validation_steps=int(ds_info.splits['train[80%:]'].num_examples/BATCH_SIZE),
    epochs=EPOCH,
    validation_data=ds_test,
    verbose=1,
    use_multiprocessing=True,
)

Epoch 1/20
581/581 [==============================] - 162s 220ms/step - loss: 0.7043 - accuracy: 0.5774 - val_loss: 0.7173 - val_accuracy: 0.5600
Epoch 2/20
581/581 [==============================] - 121s 208ms/step - loss: 0.6845 - accuracy: 0.6000 - val_loss: 0.6880 - val_accuracy: 0.5970
Epoch 3/20
581/581 [==============================] - 120s 207ms/step - loss: 0.6669 - accuracy: 0.6135 - val_loss: 2.3359 - val_accuracy: 0.4994
Epoch 4/20
581/581 [==============================] - 120s 207ms/step - loss: 0.6544 - accuracy: 0.6258 - val_loss: 0.7086 - val_accuracy: 0.5894
Epoch 5/20
581/581 [==============================] - 120s 207ms/step - loss: 0.6523 - accuracy: 0.6297 - val_loss: 1.1927 - val_accuracy: 0.5238
Epoch 6/20
581/581 [==============================] - 120s 207ms/step - loss: 0.6391 - accuracy: 0.6432 - val_loss: 0.7531 - val_accuracy: 0.5802
Epoch 7/20
581/581 [==============================] - 120s 207ms/step - loss: 0.6241 - accuracy: 0.6634 - val_loss: 0.7204 -

# resnet34

In [67]:
BATCH_SIZE = 32
EPOCH = 20

In [68]:
(ds_train, ds_test), ds_info = tfds.load(
    'cats_vs_dogs',
    split = ('train[:80%]', 'train[80%:]'),
    with_info = True,
    as_supervised = True
)

In [69]:
def normalize_and_resize_img(image, label):
    """Normalizes images: `uint8` -> `float32`."""
    image = tf.image.resize(image, [224, 224])
    return tf.cast(image, tf.float32) / 255., label

In [70]:
def apply_normalize_on_dataset(ds, is_test=False, batch_size=16):
    ds = ds.map(
        normalize_and_resize_img, 
        num_parallel_calls=1
    )
    ds = ds.batch(batch_size)
    if not is_test:
        ds = ds.repeat()
        ds = ds.shuffle(200)
    ds = ds.prefetch(tf.data.experimental.AUTOTUNE)
    return ds

In [71]:
input_tensor = Input(shape=(224, 224, 3), dtype='float32', name='input')

def conv1_layer(x):
    x = ZeroPadding2D(padding=(3,3))(x)
    x = Conv2D(64,(3,3), strides=(2,2))(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = ZeroPadding2D(padding=(1,1))(x)

    return x


In [72]:
def conv2_layer(x):
    x = MaxPooling2D((3,3), 2)(x)

    shortcut = x

    for i in range(3):
      if (i == 0):
        x = Conv2D(64, (3,3), strides=(1,1), padding='same')(x)
        x = BatchNormalization()(x)
        x = Activation('relu')(x)

        x = Conv2D(64,(3,3), strides=(1,1), padding='same')(x)
        shortcut = Conv2D(64, (1, 1), strides=(1, 1), padding='same')(shortcut)  
        x = BatchNormalization()(x)
        shortcut = BatchNormalization()(shortcut)

        x = Add()([x, shortcut])
        x = Activation('relu')(x)

        shortcut = x

      else: 
          x = Conv2D(64, (3,3), strides=(1,1), padding='same')(x)
          x = BatchNormalization()(x)
          x = Activation('relu')(x)

          x = Conv2D(64, (3,3), strides=(1,1), padding='same')(x)
          x = BatchNormalization()(x)
          x = Activation('relu')(x)


          x = Add()([x, shortcut])
          x = Activation('relu')(x)

          shortcut = x

    return x



In [73]:
def conv3_layer(x):
    shortcut = x

    for i in range(4):
        if(i==0):
            x = Conv2D(128, (3,3), strides=(2,2), padding='same')(x)
            x = BatchNormalization()(x)
            x = Activation('relu')(x)

            x = Conv2D(128, (3, 3), strides=(1,1), padding='same')(x)
            shortcut = Conv2D(128, (1, 1), strides=(2, 2), padding='same')(shortcut)
            x = BatchNormalization()(x)
            shortcut = BatchNormalization()(shortcut) 

            x = Add()([x, shortcut])
            x = Activation('relu')(x)

            shortcut = x

        else:
          x = Conv2D(128, (1,1), strides=(1,1), padding='same')(x)
          x = BatchNormalization()(x)
          x = Activation('relu')(x)

          x = Conv2D(128, (3,3), strides=(1,1), padding='same')(x)
          x = BatchNormalization()(x)

          x = Add()([x, shortcut])
          x = Activation('relu')(x)

          shortcut = x

    return x


In [74]:
def conv4_layer(x):
    shortcut = x

    for i in range(6):
        if(i == 0):
            x = Conv2D(256,(3,3), strides=(2,2), padding='same')(x)
            x = BatchNormalization()(x)
            x = Activation('relu')(x)

            x = Conv2D(256, (3,3), strides=(1,1), padding='same')(x)
            shortcut = Conv2D(256, (1, 1), strides=(2, 2), padding='same')(shortcut)
            x = BatchNormalization()(x)
            shortcut = BatchNormalization()(shortcut)


            x = Add()([x, shortcut])
            x = Activation('relu')(x)
            
            shortcut = x


        else:
            x = Conv2D(256, (3,3), strides=(1,1), padding='same')(x)
            x = BatchNormalization()(x)
            x = Activation('relu')(x)

            x = Conv2D(256, (3,3), strides=(1,1), padding='same')(x)
            x = BatchNormalization()(x)

            x = Add()([x, shortcut])
            x = Activation('relu')(x)

            shortcut = x

    return x


In [75]:
def conv5_layer(x):
    shortcut = x    
  
    for i in range(3):     
        if(i == 0):            
            x = Conv2D(512, (3, 3), strides=(2, 2), padding='same')(x)
            x = BatchNormalization()(x)
            x = Activation('relu')(x)        
            
            x = Conv2D(512, (3, 3), strides=(1, 1), padding='same')(x)
            shortcut = Conv2D(512, (1,1), strides=(2, 2), padding='same')(shortcut)
            x = BatchNormalization()(x)
            shortcut = BatchNormalization()(shortcut) 
            
 
            x = Add()([x, shortcut])  
            x = Activation('relu')(x)      
 
            shortcut = x               
        
        else:
            x = Conv2D(512, (3, 3), strides=(1, 1), padding='same')(x)
            x = BatchNormalization()(x)
            x = Activation('relu')(x)
            
            x = Conv2D(512, (3, 3), strides=(1, 1), padding='same')(x)
            x = BatchNormalization()(x)
           
            x = Add()([x, shortcut]) 
            x = Activation('relu')(x)       
 
            shortcut = x                  
 
    return x


In [76]:
x = conv1_layer(input_tensor)
x = conv2_layer(x)
x = conv3_layer(x)
x = conv4_layer(x)
x = conv5_layer(x)
 
x = GlobalAveragePooling2D()(x)
output_tensor = Dense(2, activation='softmax')(x)
 
resnet34 = Model(input_tensor, output_tensor)
resnet34.summary()


Model: "model_5"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input (InputLayer)             [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 zero_padding2d_10 (ZeroPadding  (None, 230, 230, 3)  0          ['input[0][0]']                  
 2D)                                                                                              
                                                                                                  
 conv2d_225 (Conv2D)            (None, 114, 114, 64  1792        ['zero_padding2d_10[0][0]']      
                                )                                                           

In [80]:
BATCH_SIZE = 32
EPOCH = 20

# cats_vs_dogs dataset load
(ds_train, ds_test), ds_info = tfds.load(
    'cats_vs_dogs',
    split=['train[:80%]', 'train[80%:]'],
    as_supervised=True,
    shuffle_files=True,
    with_info=True,
)
ds_train = apply_normalize_on_dataset(ds_train, batch_size=BATCH_SIZE)
ds_test = apply_normalize_on_dataset(ds_test, batch_size=BATCH_SIZE)
resnet34.compile(
    loss='sparse_categorical_crossentropy',
    optimizer=tf.keras.optimizers.SGD(learning_rate=0.01, clipnorm=1.),
    metrics=['accuracy'],
)

history_34 = resnet34.fit(
    ds_train,
    steps_per_epoch=int(ds_info.splits['train[:80%]'].num_examples/BATCH_SIZE),
    validation_steps=int(ds_info.splits['train[80%:]'].num_examples/BATCH_SIZE),
    epochs=EPOCH,
    validation_data=ds_test,
    verbose=1,
    use_multiprocessing=True,
)

Epoch 1/20
581/581 [==============================] - 108s 144ms/step - loss: 0.6410 - accuracy: 0.6397 - val_loss: 0.6020 - val_accuracy: 0.6761
Epoch 2/20
581/581 [==============================] - 74s 128ms/step - loss: 0.5707 - accuracy: 0.6987 - val_loss: 0.6719 - val_accuracy: 0.6203
Epoch 3/20
581/581 [==============================] - 74s 127ms/step - loss: 0.5171 - accuracy: 0.7421 - val_loss: 0.5904 - val_accuracy: 0.6965
Epoch 4/20
581/581 [==============================] - 74s 127ms/step - loss: 0.4671 - accuracy: 0.7737 - val_loss: 0.6570 - val_accuracy: 0.6867
Epoch 5/20
581/581 [==============================] - 74s 127ms/step - loss: 0.4199 - accuracy: 0.8053 - val_loss: 0.5722 - val_accuracy: 0.7444
Epoch 6/20
581/581 [==============================] - 74s 127ms/step - loss: 0.3669 - accuracy: 0.8364 - val_loss: 0.9030 - val_accuracy: 0.6524
Epoch 7/20
581/581 [==============================] - 74s 127ms/step - loss: 0.3056 - accuracy: 0.8678 - val_loss: 0.5079 - val_a

# Plain - resnet50

In [53]:
(ds_train, ds_test), ds_info = tfds.load(
    'cats_vs_dogs',
    split = ('train[:80%]', 'train[80%:]'),
    with_info = True,
    as_supervised = True
)


In [54]:

def normalize_and_resize_img(image, label):
    """Normalizes images: `uint8` -> `float32`."""
    image = tf.image.resize(image, [224, 224])
    return tf.cast(image, tf.float32) / 255., label

In [55]:
def apply_normalize_on_dataset(ds, is_test=False, batch_size=16):
    ds = ds.map(
        normalize_and_resize_img, 
        num_parallel_calls=1
    )
    ds = ds.batch(batch_size)
    if not is_test:
        ds = ds.repeat()
        ds = ds.shuffle(200)
    ds = ds.prefetch(tf.data.experimental.AUTOTUNE)
    return ds

In [56]:
input_tensor = Input(shape=(224, 224, 3), dtype='float32', name='input')
 
 
def conv1_layer(x):    
    x = ZeroPadding2D(padding=(3, 3))(x)
    x = Conv2D(64, (7, 7), strides=(2, 2))(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = ZeroPadding2D(padding=(1,1))(x)
 
    return x   

def conv2_layer(x):         
    x = MaxPooling2D((3, 3), 2)(x)     

    for i in range(3):
        if (i == 0):
            x = Conv2D(64, (3,3), strides=(1, 1), padding='same')(x)
            x = BatchNormalization()(x)
            x = Activation('relu')(x)

            x = Conv2D(64, (3,3), strides=(1, 1), padding='same')(x)
            x = BatchNormalization()(x) 
            x = Activation('relu')(x)
            
 
        else:
            x = Conv2D(64, (3,3), strides=(1, 1), padding='same')(x)
            x = BatchNormalization()(x)
            x = Activation('relu')(x)
            
            x = Conv2D(64, (3, 3), strides=(1, 1), padding='same')(x)
            x = BatchNormalization()(x)     
            x = Activation('relu')(x)    
    
    return x

 
def conv3_layer(x):    
    
    for i in range(4):     
        if(i == 0):            
            x = Conv2D(128, (3, 3), strides=(2, 2), padding='same')(x)
            x = BatchNormalization()(x)
            x = Activation('relu')(x)        
            
            x = Conv2D(128, (3, 3), strides=(1, 1), padding='same')(x)
            x = BatchNormalization()(x)     
            x = Activation('relu')(x)     
        
        else:
            x = Conv2D(128, (3, 3), strides=(1, 1), padding='same')(x)
            x = BatchNormalization()(x)
            x = Activation('relu')(x)
            
            x = Conv2D(128, (3, 3), strides=(1, 1), padding='same')(x)
            x = BatchNormalization()(x)       
            x = Activation('relu')(x)
            
    return x
 
 

def conv4_layer(x):  
  
    for i in range(6):     
        if(i == 0):            
            x = Conv2D(256, (3, 3), strides=(2, 2), padding='same')(x)
            x = BatchNormalization()(x)
            x = Activation('relu')(x)        

            x = Conv2D(256, (3, 3), strides=(1, 1), padding='same')(x)
            x = BatchNormalization()(x)
            x = Activation('relu')(x)          
        
        else:
            x = Conv2D(256, (3,3), strides=(1, 1), padding='same')(x)
            x = BatchNormalization()(x)
            x = Activation('relu')(x)
  
            x = Conv2D(256, (3, 3), strides=(1, 1), padding='same')(x)
            x = BatchNormalization()(x)       
            x = Activation('relu')(x)    
 
    return x
 
 

def conv5_layer(x):
    shortcut = x    
  
    for i in range(3):     
        if(i == 0):            
            x = Conv2D(512, (3, 3), strides=(2, 2), padding='same')(x)
            x = BatchNormalization()(x)
            x = Activation('relu')(x)        
 
            x = Conv2D(512, (3, 3), strides=(1, 1), padding='same')(x)
            x = BatchNormalization()(x)
            x = Activation('relu')(x)                
        
        else:
            x = Conv2D(512, (3,3), strides=(1, 1), padding='same')(x)
            x = BatchNormalization()(x)
            x = Activation('relu')(x)

            x = Conv2D(512, (3, 3), strides=(1, 1), padding='same')(x)
            x = BatchNormalization()(x)
            x = Activation('relu')(x)                    
 
    return x
 
 
  
x = conv1_layer(input_tensor)
x = conv2_layer(x)
x = conv3_layer(x)
x = conv4_layer(x)
x = conv5_layer(x)
 
x = GlobalAveragePooling2D()(x)
output_tensor = Dense(2, activation='softmax')(x)
 
resnet34_plain = Model(input_tensor, output_tensor)
resnet34_plain.summary()


Model: "model_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input (InputLayer)          [(None, 224, 224, 3)]     0         
                                                                 
 zero_padding2d_6 (ZeroPaddi  (None, 230, 230, 3)      0         
 ng2D)                                                           
                                                                 
 conv2d_159 (Conv2D)         (None, 112, 112, 64)      9472      
                                                                 
 batch_normalization_159 (Ba  (None, 112, 112, 64)     256       
 tchNormalization)                                               
                                                                 
 activation_147 (Activation)  (None, 112, 112, 64)     0         
                                                                 
 zero_padding2d_7 (ZeroPaddi  (None, 114, 114, 64)     0   

In [56]:
BATCH_SIZE = 32
EPOCH = 20

# cats_vs_dogs dataset load
(ds_train, ds_test), ds_info = tfds.load(
    'cats_vs_dogs',
    split=['train[:80%]', 'train[80%:]'],
    as_supervised=True,
    shuffle_files=True,
    with_info=True,
)
ds_train = apply_normalize_on_dataset(ds_train, batch_size=BATCH_SIZE)
ds_test = apply_normalize_on_dataset(ds_test, batch_size=BATCH_SIZE)
resnet34.compile(
    loss='sparse_categorical_crossentropy',
    optimizer=tf.keras.optimizers.SGD(learning_rate=0.01, clipnorm=1.),
    metrics=['accuracy'],
)

history_34 = resnet50.fit(
    ds_train,
    steps_per_epoch=int(ds_info.splits['train[:80%]'].num_examples/BATCH_SIZE),
    validation_steps=int(ds_info.splits['train[80%:]'].num_examples/BATCH_SIZE),
    epochs=EPOCH,
    validation_data=ds_test,
    verbose=1,
    use_multiprocessing=True,
)

# plain - resnet34

In [77]:
# TensorFlow and tf.keras
import tensorflow as tf
from tensorflow import keras
from keras import models, layers
from keras import Input
from keras.models import Model, load_model
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers, initializers, regularizers, metrics
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.layers import BatchNormalization, Conv2D, Activation, Dense, GlobalAveragePooling2D, MaxPooling2D, ZeroPadding2D, Add

import os
import matplotlib.pyplot as plt
import numpy as np
import math

import tensorflow_datasets as tfds

In [78]:
(ds_train, ds_test), ds_info = tfds.load(
    'cats_vs_dogs',
    split = ('train[:80%]', 'train[80%:]'),
    with_info = True,
    as_supervised = True
)

In [79]:
def normalize_and_resize_img(image, label):
    """Normalizes images: `uint8` -> `float32`."""
    image = tf.image.resize(image, [224, 224])
    return tf.cast(image, tf.float32) / 255., label

In [80]:
def apply_normalize_on_dataset(ds, is_test=False, batch_size=16):
    ds = ds.map(
        normalize_and_resize_img, 
        num_parallel_calls=1
    )
    ds = ds.batch(batch_size)
    if not is_test:
        ds = ds.repeat()
        ds = ds.shuffle(200)
    ds = ds.prefetch(tf.data.experimental.AUTOTUNE)
    return ds

In [81]:
input_tensor = Input(shape=(224, 224, 3), dtype='float32', name='input')
 

In [82]:
 def conv1_layer(x):    
    x = ZeroPadding2D(padding=(3, 3))(x)
    x = Conv2D(64, (7, 7), strides=(2, 2))(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = ZeroPadding2D(padding=(1,1))(x)
 
    return x   


In [83]:
def conv2_layer(x):         
    x = MaxPooling2D((3, 3), 2)(x)     

    for i in range(3):
        if (i == 0):
            x = Conv2D(64, (3,3), strides=(1, 1), padding='same')(x)
            x = BatchNormalization()(x)
            x = Activation('relu')(x)

            x = Conv2D(64, (3,3), strides=(1, 1), padding='same')(x)
            x = BatchNormalization()(x) 
            x = Activation('relu')(x)
            
 
        else:
            x = Conv2D(64, (3,3), strides=(1, 1), padding='same')(x)
            x = BatchNormalization()(x)
            x = Activation('relu')(x)
            
            x = Conv2D(64, (3, 3), strides=(1, 1), padding='same')(x)
            x = BatchNormalization()(x)     
            x = Activation('relu')(x)    
    
    return x


In [84]:
def conv3_layer(x):    
    
    for i in range(4):     
        if(i == 0):            
            x = Conv2D(128, (3, 3), strides=(2, 2), padding='same')(x)
            x = BatchNormalization()(x)
            x = Activation('relu')(x)        
            
            x = Conv2D(128, (3, 3), strides=(1, 1), padding='same')(x)
            x = BatchNormalization()(x)     
            x = Activation('relu')(x)     
        
        else:
            x = Conv2D(128, (3, 3), strides=(1, 1), padding='same')(x)
            x = BatchNormalization()(x)
            x = Activation('relu')(x)
            
            x = Conv2D(128, (3, 3), strides=(1, 1), padding='same')(x)
            x = BatchNormalization()(x)       
            x = Activation('relu')(x)
            
    return x
 
 


In [85]:
def conv4_layer(x):  
  
    for i in range(6):     
        if(i == 0):            
            x = Conv2D(256, (3, 3), strides=(2, 2), padding='same')(x)
            x = BatchNormalization()(x)
            x = Activation('relu')(x)        

            x = Conv2D(256, (3, 3), strides=(1, 1), padding='same')(x)
            x = BatchNormalization()(x)
            x = Activation('relu')(x)          
        
        else:
            x = Conv2D(256, (3,3), strides=(1, 1), padding='same')(x)
            x = BatchNormalization()(x)
            x = Activation('relu')(x)
  
            x = Conv2D(256, (3, 3), strides=(1, 1), padding='same')(x)
            x = BatchNormalization()(x)       
            x = Activation('relu')(x)    
 
    return x
 

In [86]:
def conv5_layer(x):
    shortcut = x    
  
    for i in range(3):     
        if(i == 0):            
            x = Conv2D(512, (3, 3), strides=(2, 2), padding='same')(x)
            x = BatchNormalization()(x)
            x = Activation('relu')(x)        
 
            x = Conv2D(512, (3, 3), strides=(1, 1), padding='same')(x)
            x = BatchNormalization()(x)
            x = Activation('relu')(x)                
        
        else:
            x = Conv2D(512, (3,3), strides=(1, 1), padding='same')(x)
            x = BatchNormalization()(x)
            x = Activation('relu')(x)

            x = Conv2D(512, (3, 3), strides=(1, 1), padding='same')(x)
            x = BatchNormalization()(x)
            x = Activation('relu')(x)                    
 
    return x
 
 

In [87]:
x = conv1_layer(input_tensor)
x = conv2_layer(x)
x = conv3_layer(x)
x = conv4_layer(x)
x = conv5_layer(x)
 
x = GlobalAveragePooling2D()(x)
output_tensor = Dense(2, activation='softmax')(x)
 
resnet34_plain = Model(input_tensor, output_tensor)
resnet34_plain.summary()

Model: "model_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input (InputLayer)          [(None, 224, 224, 3)]     0         
                                                                 
 zero_padding2d_12 (ZeroPadd  (None, 230, 230, 3)      0         
 ing2D)                                                          
                                                                 
 conv2d_262 (Conv2D)         (None, 112, 112, 64)      9472      
                                                                 
 batch_normalization_262 (Ba  (None, 112, 112, 64)     256       
 tchNormalization)                                               
                                                                 
 activation_248 (Activation)  (None, 112, 112, 64)     0         
                                                                 
 zero_padding2d_13 (ZeroPadd  (None, 114, 114, 64)     0   

# 회고 

- padding
    - valid ; 유효한 영역만 출력된다. 따라서 출력 이미지 사이즈는 입력 사이즈보다 작다.
    - same ; 출력 이미지 사이즈가 입력 이미지 사이즈와 동일하다. 